In [4]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 

from utils.torch_utils import get_default_device

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from datasets import load_cifar10

batch_size = 16
num_workers = 2

(trainloader, validloader, testloader), classes = load_cifar10(batch_size, num_workers=num_workers, train_ratio=0.8)
print("CIFAR {} classes: {}".format(len(classes), classes))
print("Train size: {}, Valid size: {}, Test size: {}".format(len(trainloader), len(validloader), len(testloader)))

Files already downloaded and verified
Files already downloaded and verified
CIFAR 10 classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Train size: 2500, Valid size: 625, Test size: 625


In [8]:
from resnet_cifar import ResnetCifar

model = ResnetCifar(n_classes=len(classes))
device = get_default_device()
model = model.to(device)

loss_func = torch.nn.CrossEntropyLoss()

In [20]:
from trainer import Trainer

num_epochs = 10
log_interval = 2

trainer = Trainer(model, trainloader, validloader, testloader, loss_func, lr=0.001, optimizer='adam')
model, learn_hists, best_epoch = trainer.train_model(num_epochs=1)
# plt.plot(np.arange(num_epochs), loss_history)
    


Epoch 0/0
----------


 17%|█▋        | 436/2500 [00:12<00:45, 44.92it/s, loss=3.03e+3, acc=tensor(0.1593, device='mps:0')]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x115ce91c0>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dpo/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dpo/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dpo/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/dpo/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/hom

KeyboardInterrupt: 